<a href="https://colab.research.google.com/github/kushArya/CS382/blob/main/Best_Edit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
alpha = "abcdefghijklmnopqrstuvwxyz"

def kd_matrix():
    
    # For each letter key, give its neighbors on the keyboard.
    # The first letter is the key, and the remaining letters are the neighbors.
    # This was easy for me to type into this code.
    # Below I turn this into a dictionary that mimics an adjacency list representation.
    kd_adj = [
        "qwa","wesaq","erdsw","rtfde","tygfr","yuhgt","uijhy","iokju","oplki","plo",
        "aqwsz","sawedxz","dserfcx","fdrtgvc","gftyhbv","hgyujnb","jhuikmn","kjiolm","lkop",
        "zasx","xzsdc","cxdfv","vcfgb","bvghn","nbhjm","mnjk"
        ]

    # Convert to a dictionary with entries 'a' thru 'z' corresponding to keys.
    # Each key's entry has a string of that key's neighbors.
    adj = {}
    for k_ns in kd_adj:
        k  = k_ns[0]
        ns = k_ns[1:]
        adj[k] = ns

    # Build a matrix of keyboard distances, represented as a dictionary of dictionaries.
    # We do this by running BFS 26 times on the graph defined above.
    # The result is a Python object `mtx` where mtx[k1][k2] gives the distance between
    # keys k1 and k2 on the keyboard.
    # For example, mtx['f','f'] is 0, mtx['f','g'] is 1, and mtx['f','u'] is 3. 

    mtx = {}
    for s in alpha:

        # Perform a BFS seeded with the key s.
        Q = []
        head = 0
        dist = {s:0}
        Q.append(s)
        # Create a dictionary of distances from s to every key.
        while head < len(Q):
            u = Q[head]
            head += 1
            for v in adj[u]:
                if v not in dist:
                    dist[v] = dist[u] + 1
                    Q.append(v)

        # Add these distances as the entry for s in the matrix.
        mtx[s] = dist

    return mtx

#            
# Compute a matrix of substitution costs useful for spell-checking.
# The expression
# 
#   ks_mtx[k1][k2]
#
# will give you a distance between alphabetic keys k1 and k2 on the
# keyboard. Here, k1 and k2 are length 1 character strings 'a' thru 'z'.
#
# E.g. kd_mtx['f','f'] is 0, kd_mtx['f','g'] is 1, and kd_mtx['f','u'] is 3. 
#

kd_mtx = kd_matrix()   

In [11]:
import numpy as np
d_cost = 3
i_cost = 3
def bestEdit(x, y):
  why = {}                                      #each element of BE gets a why so we don't need anymore calculations after determining BE[n-1,m-1]
  n, m = len(x), len(y)
  BE = np.zeros(shape=(n,m))

  BE[0,0] = 0
  for i in range(n):
    for j in range(m):
      why[BE[i,j]] = ''                         #initializing all keys so we can do string addition and backwards references without error
  for i in range(1, n):
    BE[i,0] = BE[i-1, 0] + d_cost
    why[BE[i,0]] = why[BE[i-1, 0]] + 'deleted: ' + x[i]
  for j in range(1, m):
    BE[0,j] = BE[0, j-1] + i_cost
    why[BE[0,j]] = why[BE[i, j-1]] + '\n inserted: ' + y[j]
  for i in range(1, n):
    for j in range(1, m):
      ins = BE[i,j-1] + i_cost
      cs = BE[i-1, j-1] + kd_mtx[x[i]][y[j]]    #the cost of substitution is determined by the key distances
      de = BE[i-1, j] + d_cost
      t = de + 1                                #jank way to make sure t isn't the min when transpose isn't viable
      if x[i]==y[j-1] and x[i-1]==y[j]:
        t = BE[i-2, j-2] + 1
      a = min(ins, cs, de, t)                   #assgining a variable bc used lots in the why
      BE[i,j] = a
      if a == ins: why[BE[i,j]] = why[BE[i, j-1]] + '\n inserted: ' + y[j]                        #it might be optimal to make a dictionary or tuple or something, but this works fine
      elif a == cs: why[BE[i,j]] = why[BE[i-1, j-1]] + '\n substituted: ' + x[i] + ' for ' + y[j]
      elif a == de: why[BE[i,j]] = why[BE[i-1, j]] + '\n deleted: ' + x[i]
      elif a == t: why[BE[i,j]] = why[BE[i-2, j-2]] + '\n twiddled: ' + x[i-1] + ' and ' + x[i] 

  print(BE)
  print(why[BE[n-1,m-1]])
  return BE[n-1,m-1]

bestEdit('abdc', 'bacd')

[[0. 3. 6. 9.]
 [3. 1. 4. 7.]
 [6. 4. 2. 4.]
 [9. 7. 4. 2.]]

 transposed: a and b
 transposed: d and c


2.0